In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
cd gdrive/MyDrive/python/data

/content/gdrive/MyDrive/python/data


In [ ]:
!pip install -r requirements.txt #이 프로젝트를 하기 위해 설치해야 하는 library들이 requiremets에 들어있습니다.

Processing /dlib-19.18.0-cp37-cp37m-linux_x86_64.whl
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/dlib-19.18.0-cp37-cp37m-linux_x86_64.whl'



# Selfie to Anime Project

데이터셋: https://www.kaggle.com/arnaud58/selfie2anime
download를 통해 다운받고, hw_leaf를 참고해 폴더를 업로드

 **Code 0. Import Library**

In [ ]:
import glob    # 파일들의 리스트를 뽑아 경로명을 이용
import random    # 랜덤 관련 함수
import os

from torch.utils.data import Dataset, DataLoader    # dataset : 샘플과 정답(label) 저장 / dataloader : dataset을 샘플에 쉽게 접근하도록
from PIL import Image    # Python Imaging Library : 이미지 분석 및 처리
import torchvision.transforms as transforms    # 다양한 이미지 변환 기능
import sys
import torch.nn as nn    # 신경망 생성& 정의
import torch.nn.functional as F    # 신경망 생성& 정의
import torch

import os
import numpy as np
import math
import itertools    # 조합, 순열 함수
import datetime
import time

from torchvision.utils import save_image, make_grid    # save_image : 이미지 저장 / make_grid : 이미지 학습 때 중간중간의 결과 확인
from torchvision import datasets    # torchvision에서 제공하는 데이터 셋 사용
from torch.autograd import Variable    # 각 tensor 값을 볼 수 있는 data, 미분을 보는 grad, backward를 통한 미분 계산 함수 정보인 grad_fn

**Code 1. Dataset**

In [ ]:
def to_rgb(image):
    rgb_image = Image.new("RGB", image.size)    # 이미지를 새로 생성. Imgae.new(mode, size, color) / 여기서 mode는 RGB로 3*8-bit pixels, true color. size는 256*256. 즉 사이즈는 유지
    rgb_image.paste(image)    # 이미지끼리 이어붙임
    return rgb_image

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, unaligned=False, mode="train"):
        self.transform = transforms.Compose(transforms_)    # transforms_에 따라 이미지 변환
        self.unaligned = unaligned    # 학습 이미지쌍을 무작위로 고를지 고정시킬지 정함. 여기선 False(고정)인 경우는 없음
        # train 모드일 때는 trainA, trainB에 있는 디렉토리에서 이미지를 불러옵니다. 
        if mode=="train":    # 학습모드일 땐 train 폴더 내의 이미지 사용
            # glob 함수로 trainA 디렉토리의 이미지의 목록을 불러옵니다. 
            self.files_A = sorted(glob.glob(os.path.join(root, "trainA") + "/*.*"))    # 경로의 trainA 폴더 내 모든 파일 목록을 가져옴
            self.files_B = sorted(glob.glob(os.path.join(root, "trainB") + "/*.*"))    # 경로의 trainB 폴더 내 모든 파일 목록을 가져옴
        else:    # 학습모드가 아닐 땐 test 폴더 내 이미지 사용
            self.files_A = sorted(glob.glob(os.path.join(root, "testA") + "/*.*"))    # 경로의 testA 폴더 내 모든 파일 목록을 가져옴
            self.files_B = sorted(glob.glob(os.path.join(root, "testB") + "/*.*"))    # 경로의 testB 폴더 내 모든 파일 목록을 가져옴

    def __getitem__(self, index):
        # index값으로 이미지의목록 중 이미지 하나를 불러옵니다. 
        image_A = Image.open(self.files_A[index % len(self.files_A)])
        # unaligned 변수로 학습할 Pair를 랜덤으로 고릅니다.
        if self.unaligned:
            image_B = Image.open(self.files_B[random.randint(0, len(self.files_B) - 1)])
        else:
            image_B = Image.open(self.files_B[index % len(self.files_B)])

        # Convert grayscale images to rgb
        # 흑백이미지의 경우 RGB 형태로 변경
        if image_A.mode != "RGB":
            image_A = to_rgb(image_A)
        if image_B.mode != "RGB":
            image_B = to_rgb(image_B)

        item_A = self.transform(image_A)    # transform 진행한 이미지 저장
        item_B = self.transform(image_B)    # 위와 동일
        return {"A": item_A, "B": item_B}

    def __len__(self):
        return max(len(self.files_A), len(self.files_B))    # trainA, trainB 폴더 내 파일 개수 반환

**Code 2. Generator & Discriminator**

In [ ]:
def weights_init_normal(m):    # 가중치 초기화 함수. 클래스(layer)의 종류에 따라 가중치 초기화를 다르게 진행
    classname = m.__class__.__name__    # 클래스 이름 참조
    if classname.find("Conv") != -1:    # Conv 클래스의 경우
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)    # 가중치(tensor형)에 정규분포 평균 0, 표준편차 0.02를 따르는 값을 넣음
        if hasattr(m, "bias") and m.bias is not None:    # bias 변수가 있고 None 값이 아니라면
            torch.nn.init.constant_(m.bias.data, 0.0)    # bias값을 0으로
    elif classname.find("BatchNorm2d") != -1:    # BatchNorm2d 클래스의 경우
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)    # 위와 같지만 평균 1
        torch.nn.init.constant_(m.bias.data, 0.0)    # bias는 모두 0값으로

In [ ]:
class ResidualBlock(nn.Module):    # 내부 layer인 Residual Block 구현 : 기울기 소실 문제를 해결하기 위함
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()    # ResidualBlock 클래스를 상속. 여기선 nn.Sequential을 상속받음. 이로 forward method를 작성하지 않아도 됨

        self.block = nn.Sequential(    # layer 형태보다 가독성 뛰어남
            nn.ReflectionPad2d(1),    # 요건 밑을 보는 것이 편할듯... 입력 이미지의 패딩을 반사기법 사용. 더 자연스러운 이미지 생성 도움
            nn.Conv2d(in_features, in_features, 3),    # in&out 채널은 그대로, kernel은 3
            nn.InstanceNorm2d(in_features),    # Instance들을 개별로 Normalize. Batch Normalization과는 차이
            nn.ReLU(inplace=True),    # Relu. inplace=True면 input으로 들어온 것 자체를 수정. 메모리 usage가 조금 사용되지만 input을 제거
            nn.ReflectionPad2d(1),    # 위와 같음
            nn.Conv2d(in_features, in_features, 3),    # 위와 같음
            nn.InstanceNorm2d(in_features),    # 위와 같음
        )

    def forward(self, x):
        return x + self.block(x)    # ResidaulBlock 실행 결과를 더함

In [ ]:
xxx = torch.arange(16, dtype=torch.float).reshape(1, 1, 4, 4)
xxx

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [ ]:
mmm = nn.ReflectionPad2d(1)
mmm(xxx)

tensor([[[[ 5.,  4.,  5.,  6.,  7.,  6.],
          [ 1.,  0.,  1.,  2.,  3.,  2.],
          [ 5.,  4.,  5.,  6.,  7.,  6.],
          [ 9.,  8.,  9., 10., 11., 10.],
          [13., 12., 13., 14., 15., 14.],
          [ 9.,  8.,  9., 10., 11., 10.]]]])

In [ ]:
mmm = nn.ReflectionPad2d(2)
mmm(xxx)

tensor([[[[10.,  9.,  8.,  9., 10., 11., 10.,  9.],
          [ 6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.],
          [ 2.,  1.,  0.,  1.,  2.,  3.,  2.,  1.],
          [ 6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.],
          [10.,  9.,  8.,  9., 10., 11., 10.,  9.],
          [14., 13., 12., 13., 14., 15., 14., 13.],
          [10.,  9.,  8.,  9., 10., 11., 10.,  9.],
          [ 6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.]]]])

In [ ]:
mmm = nn.ReflectionPad2d(3)
mmm(xxx)

tensor([[[[15., 14., 13., 12., 13., 14., 15., 14., 13., 12.],
          [11., 10.,  9.,  8.,  9., 10., 11., 10.,  9.,  8.],
          [ 7.,  6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.,  4.],
          [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  2.,  1.,  0.],
          [ 7.,  6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.,  4.],
          [11., 10.,  9.,  8.,  9., 10., 11., 10.,  9.,  8.],
          [15., 14., 13., 12., 13., 14., 15., 14., 13., 12.],
          [11., 10.,  9.,  8.,  9., 10., 11., 10.,  9.,  8.],
          [ 7.,  6.,  5.,  4.,  5.,  6.,  7.,  6.,  5.,  4.],
          [ 3.,  2.,  1.,  0.,  1.,  2.,  3.,  2.,  1.,  0.]]]])

In [ ]:
class GeneratorResNet(nn.Module):
    def __init__(self, input_shape, num_residual_blocks):
        super(GeneratorResNet, self).__init__()    # 역시나 상속

        channels = input_shape[0]    # 채널의 수 저장

        # Initial convolution block
        out_features = 64    # output 채널의 수 = 64
        model = [
            nn.ReflectionPad2d(channels),    # 리플렉션패딩.. 채널수만큼
            nn.Conv2d(channels, out_features, 7),    # 채널의 수 input, 64 채널 output, kernel=7
            nn.InstanceNorm2d(out_features),    # batch 단위 normalize
            nn.ReLU(inplace=True),    # relu 진행
        ]
        in_features = out_features    # 이젠 채널의 in&out값 통일

        # Downsampling
        for _ in range(2):    # 2번 반복
            out_features *= 2    # out channel 값에 2씩 곱함.. 각각 128, 256
            model += [    # 모델에 레이어를 한번씩 더함
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),    # 1step : 3, 128, 3, 2, 1 / 2step : 128, 256, 3, 2, 1
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features    # in & out channel 값 같게
        # 이미지는 어떻게...?
        # (1, 3, 256, 256) -> (1, 128, 128, 128) -> (1, 256, 64, 64)

        # Residual blocks
        for _ in range(num_residual_blocks):    # residual blocks 수 만큼
            model += [ResidualBlock(out_features)]    # 모델에 ResidualBlock 함수를 더함 (out_features = 256)

        # Upsampling
        for _ in range(2):    # 2번 반복
            out_features //= 2    # out channel 값을 2씩 나눔. 각각 128, 64
            model += [    # 모델에 레이어를 한번씩 더함
                nn.Upsample(scale_factor=2),    # 이미지를 2배씩 확대
                nn.Conv2d(in_features, out_features, 3, stride=1, padding=1),   # 1step : 256, 128, 3, 1, 1 / 2step : 128, 64, 3, 1, 1
                nn.InstanceNorm2d(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features    # in & out channel 값 같게
        # 이미지는 어떻게...?
        # (1, 256, 64, 64) -> (1, 128, 128, 128) -> (1, 64, 256, 256)

        # Output layer : 출력 레이어
        model += [nn.ReflectionPad2d(channels), nn.Conv2d(out_features, channels, 7), nn.Tanh()]    # 모델 마지막에 리플렉트 패딩, conv(64, 64, 7), tanh로 활성화
        # 초기 input 이미지의 채널값은 3이었으므로 channels = 3. 이미지는 그럼... (256+3*2) - 7+1 = 256. 즉, 입력 이미지와 출력 이미지의 크기는 동일하다.

        self.model = nn.Sequential(*model)    # 모델의 sequential화...? 밑에 확인

    def forward(self, x):
        return self.model(x)

In [ ]:
model = [
            nn.ReflectionPad2d(channels),
            nn.Conv2d(channels, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        ]

NameError: ignored

In [ ]:
model

NameError: ignored

In [ ]:
model = nn.Sequential(*model)
model

NameError: ignored

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()    # 상속받자!

        channels, height, width = input_shape    # 이미지의 채널, 세로길이, 가로길이

        # Calculate output shape of image discriminator (PatchGAN)
        self.output_shape = (1, height // 2 ** 4, width // 2 ** 4)    # output 이미지의 크기... 채널 1개, 가로&세로길이 16으로 나눈 값. 즉 (1, 16, 16)

        def discriminator_block(in_filters, out_filters, normalize=True):    # 이미지 다운샘플링 함수 정의
            """Returns downsampling layers of each discriminator block"""
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]    # layer는 conv. input, output, kernel = 4, stride = 2, padding = 1
            if normalize:    # normalize해야하면
                layers.append(nn.InstanceNorm2d(out_filters))    # layer에 instancenorm2d 붙이고
            layers.append(nn.LeakyReLU(0.2, inplace=True))    # leakyReLU를 붙이자
            return layers

        self.model = nn.Sequential(    # Sequential화 시켜주자
            *discriminator_block(channels, 64, normalize=False),    # 위의 discriminator에 output 64, normalize 필요 레이어를 먼저 추가
            *discriminator_block(64, 128),    # 이어서 input 64, output 128 레이어 추가
            *discriminator_block(128, 256),    # input 128, output 256
            *discriminator_block(256, 512),    # input 256, output512
            nn.ZeroPad2d((1, 0, 1, 0)),    # 0으로 이루어진 패딩을 왼쪽과 위쪽에만 1개씩 붙임
            nn.Conv2d(512, 1, 4, padding=1)    # 마지막 conv layer. 512개의 input을 1개의 output으로, kernel=4, padding=1
        )
        # 위의 결과로 나온 이미지의 크기는?? : (1, 3, 256, 256) -> (1, 64, 128, 128) -> (1, 128, 64, 64) -> (1, 256, 32, 32) -> (1, 512, 16, 16) -> (1, 512, 17, 17) -> (1, 1, 16, 16)

    def forward(self, img):
        return self.model(img)

In [ ]:
xxx = torch.randn(1, 1, 3, 3)
xxx

tensor([[[[ 0.9125, -1.6312, -1.7842],
          [ 0.3164, -0.8926, -0.3524],
          [ 1.6352,  0.3519,  0.7207]]]])

In [ ]:
mmm = nn.ZeroPad2d((1, 0, 1, 0))
mmm(xxx)

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.9125, -1.6312, -1.7842],
          [ 0.0000,  0.3164, -0.8926, -0.3524],
          [ 0.0000,  1.6352,  0.3519,  0.7207]]]])

**Code 3. Training**

In [ ]:
dataset_name="selfie2anime"
channels = 3
img_height = 256
img_width = 256
n_residual_blocks=9
lr=0.0002
b1=0.5
b2=0.999
#n_epochs=200
n_epochs=20
init_epoch=0
#decay_epoch=100
decay_epoch=10
lambda_cyc=10.0
lambda_id=5.0    # Identity loss에 대한 람다값. 클수록 본래의 색감 유지
n_cpu=8
batch_size=1
sample_interval=100
checkpoint_interval=5

In [ ]:
class opt :
  dataset_name="selfie2anime"
  channels = 3
  img_height = 256
  img_width = 256
  n_residual_blocks=9
  lr=0.0002
  b1=0.5
  b2=0.999
  #n_epochs=200
  n_epochs=20
  init_epoch=0
  #decay_epoch=100
  decay_epoch=10
  lambda_cyc=10.0
  lambda_id=5.0
  n_cpu=8
  batch_size=1
  sample_interval=100
  checkpoint_interval=5

In [ ]:
# Create sample and checkpoint directories
os.makedirs("images/%s" % dataset_name, exist_ok=True)    # 이미지를 저장할 경로 존재하면 따로 생성하지 않음
os.makedirs("saved_models/%s" % dataset_name, exist_ok=True)    # 모델을 저장할 경로 존재하면 따로 생성하지 않음

In [ ]:
# Losses
criterion_GAN = torch.nn.MSELoss()    # GAN의 손실함수는 MSE
criterion_cycle = torch.nn.L1Loss()    # 사이클의 손실함수는 L1 Loss
criterion_identity = torch.nn.L1Loss()    # 아이덴티티의 손실함수는 L1 Loss

In [ ]:
input_shape = (channels, img_height, img_width)    # input 데이터 shape

# Initialize generator and discriminator
G_AB = GeneratorResNet(input_shape, n_residual_blocks)    # 사람들 이미지로 애니이미지를 만드는 generator
G_BA = GeneratorResNet(input_shape, n_residual_blocks)    # 애니이미지로 사람들 이미지를 만드는 generator
D_A = Discriminator(input_shape)    # 사람들 이미지 discriminator
D_B = Discriminator(input_shape)    # 애니 이미지 discriminator

In [ ]:
cuda = torch.cuda.is_available()    # cuda 사용 여부 확인

if cuda:    # cuda 되면 모두 cuda로
    G_AB = G_AB.cuda()
    G_BA = G_BA.cuda()
    D_A = D_A.cuda()
    D_B = D_B.cuda()
    criterion_GAN.cuda()
    criterion_cycle.cuda()
    criterion_identity.cuda()

In [ ]:
# Initialize weights
G_AB.apply(weights_init_normal)    # 사람=>애니 generator 가중치 초기화
G_BA.apply(weights_init_normal)    # 애니=>사람 generator 가중치 초기화
D_A.apply(weights_init_normal)    # 사람 discriminator 가중치 초기화
D_B.apply(weights_init_normal)    # 애니 discriminator 가중치 초기화

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  )
)

In [ ]:
# Optimizers
# generator, discriminator 모두 Adam 사용
optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1, b2)
)
optimizer_D_A = torch.optim.Adam(D_A.parameters(), lr=lr, betas=(b1, b2))
optimizer_D_B = torch.optim.Adam(D_B.parameters(), lr=lr, betas=(b1, b2))

In [ ]:
class LambdaLR:    # Learning Rate를 Decay할 Epcoh 설정
    def __init__(self, n_epochs, offset, decay_start_epoch):
        assert (n_epochs - decay_start_epoch) > 0, "Decay must start before the training session ends!"    # epoch 값이 decay_start_epcoh 값보다 커야함
        self.n_epochs = n_epochs
        self.offset = offset
        self.decay_start_epoch = decay_start_epoch

    def step(self, epoch):
        return 1.0 - max(0, epoch + self.offset - self.decay_start_epoch) / (self.n_epochs - self.decay_start_epoch)

In [ ]:
# Learning rate update schedulers
# 학습률 조정. 학습률에 lr_lambda 값을 곱해 감소시킴
lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(
    optimizer_G, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_A, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(
    optimizer_D_B, lr_lambda=LambdaLR(n_epochs, init_epoch, decay_epoch).step
)

In [ ]:
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor    # GPU 연산을 위한 자료형 변환

In [ ]:
class ReplayBuffer:    # torch 변수가 requires_grad=True로 설정되어 있다면 매 연산마다 gradient를 저장하므로 이미지를 따로 저장
    def __init__(self, max_size=50):
        assert max_size > 0, "Empty buffer or trying to create a black hole. Be careful."    # max_size가 0보다 커야만 진행
        self.max_size = max_size
        self.data = []

    def push_and_pop(self, data):
        to_return = []
        for element in data.data:
            element = torch.unsqueeze(element, 0)    # element의 맨 앞에 차원 추가
            if len(self.data) < self.max_size:    # data의 크기가 max_size보다 작으면
                self.data.append(element)    # data에 element값 붙임
                to_return.append(element)    # to_return 값에도 element 붙임
            else:
                if random.uniform(0, 1) > 0.5:    # 0과 1사이의 임의의 실수를 뽑았을 때 0.5보다 크면
                    i = random.randint(0, self.max_size - 1)    # 0과 max_size-1값 사이 임의의 실수를 i로 지정하고
                    to_return.append(self.data[i].clone())    # data의 i번째 값을 복사하여 to_return에 추가
                    self.data[i] = element    # data의 i번째 값은 element로 대체
                else:
                    to_return.append(element)    # 0.5보다 작았을 땐 element 값을 to_return에 추가
        return Variable(torch.cat(to_return))    # tensor list를 한번에 tensor로 변경한 후 Variable 생성. variable은 미분을 진행할 수 있도록 함

In [ ]:
# Buffers of previously generated samples
fake_A_buffer = ReplayBuffer()    # 이전 생성 샘플(사람)들의 버퍼
fake_B_buffer = ReplayBuffer()    # 이전 생성 샘플(애니)들의 버퍼

In [ ]:
# Image transformations
transforms_ = [
    transforms.Resize(int(img_height * 1.12), Image.BICUBIC),    # 이미지 크기 변환. 이미지크기*1.12, 리샘플링 방법은 BICUBIC
    transforms.RandomCrop((img_height, img_width)),    # 이미지를 자름. 256*256
    transforms.RandomHorizontalFlip(),    # 이미지 랜덤하게 좌우반전
    transforms.ToTensor(),    # 텐서화
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),    # 평균&표준편차별 normalization 진행
]

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:288: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


In [ ]:
# Training data loader
dataloader = DataLoader(
    ImageDataset("./images/%s" % dataset_name, transforms_=transforms_, unaligned=True),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)
# Test data loader
val_dataloader = DataLoader(
    ImageDataset("./images/%s" % dataset_name, transforms_=transforms_, unaligned=True, mode="test"),
    batch_size=5,
    shuffle=True,
    num_workers=1,
)

ValueError: ignored

In [ ]:
def sample_images(batches_done):
    """Saves a generated sample from the test set"""
    imgs = next(iter(val_dataloader))    # test data의 목록에서 파일을 하나씩 불러오면서 imgs에 저장
    G_AB.eval()    # generator 사람=>애니 평가모드 전환
    G_BA.eval()    # generator 사람+애니 평가모드 전환
    real_A = Variable(imgs["A"].type(Tensor))    # 사람 이미지를 텐서화하여 variable 생성
    fake_B = G_AB(real_A)    # 사람 이미지를 가지고 애니 이미지 생성
    real_B = Variable(imgs["B"].type(Tensor))    # 애니 이미지를 텐서화하여 variable 생성
    fake_A = G_BA(real_B)    # 애니 이미지를 가지고 사람 이미지 생성
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True)    # 실제 사람 이미지 5개를 가로로 붙임
    real_B = make_grid(real_B, nrow=5, normalize=True)    # 실제 애니 이미지 5개를 가로로 붙임
    fake_A = make_grid(fake_A, nrow=5, normalize=True)    # 생성 애니 이미지 5개를 가로로 붙임
    fake_B = make_grid(fake_B, nrow=5, normalize=True)    # 생성 사람 이미지 5개를 가로로 붙임
    # Arange images along y-axis
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A), 1)    # 위에서 붙인 이미지들을 세로로 붙임
    save_image(image_grid, "images/%s/%s.png" % (dataset_name, batches_done), normalize=False)    # 이미지 저장

In [ ]:
l = [1,2,3,4,5]
ir = iter(l)

print(next(ir))
print(next(ir))
print(next(ir))
print(next(ir))
print(next(ir))

1
2
3
4
5


In [ ]:
real_A = Variable(batch["A"].type(Tensor))

In [ ]:
batch["A"].shape

torch.Size([1, 3, 256, 256])

In [ ]:
batch['A']

tensor([[[[-0.0510, -0.0353, -0.0118,  ...,  0.6941,  0.6941,  0.6941],
          [-0.0510, -0.0353, -0.0118,  ...,  0.7020,  0.7020,  0.7020],
          [-0.0431, -0.0353, -0.0039,  ...,  0.6941,  0.6941,  0.6941],
          ...,
          [-0.6471, -0.6471, -0.6627,  ..., -0.5529, -0.5373, -0.5373],
          [-0.6549, -0.6549, -0.6706,  ..., -0.5529, -0.5373, -0.5294],
          [-0.6549, -0.6627, -0.6784,  ..., -0.5529, -0.5451, -0.5373]],

         [[-0.0588, -0.0431, -0.0196,  ...,  0.7961,  0.7961,  0.7961],
          [-0.0588, -0.0431, -0.0196,  ...,  0.8039,  0.8039,  0.8039],
          [-0.0431, -0.0353, -0.0118,  ...,  0.8039,  0.8039,  0.8039],
          ...,
          [-0.8039, -0.8118, -0.8431,  ..., -0.5059, -0.4902, -0.4902],
          [-0.8118, -0.8196, -0.8510,  ..., -0.5137, -0.4980, -0.4902],
          [-0.8118, -0.8275, -0.8588,  ..., -0.5137, -0.5059, -0.4980]],

         [[-0.0118, -0.0039,  0.0275,  ...,  0.6627,  0.6627,  0.6627],
          [-0.0039,  0.0118,  

In [ ]:
real_A.shape

torch.Size([1, 3, 256, 256])

In [ ]:
real_A

tensor([[[[-0.0510, -0.0353, -0.0118,  ...,  0.6941,  0.6941,  0.6941],
          [-0.0510, -0.0353, -0.0118,  ...,  0.7020,  0.7020,  0.7020],
          [-0.0431, -0.0353, -0.0039,  ...,  0.6941,  0.6941,  0.6941],
          ...,
          [-0.6471, -0.6471, -0.6627,  ..., -0.5529, -0.5373, -0.5373],
          [-0.6549, -0.6549, -0.6706,  ..., -0.5529, -0.5373, -0.5294],
          [-0.6549, -0.6627, -0.6784,  ..., -0.5529, -0.5451, -0.5373]],

         [[-0.0588, -0.0431, -0.0196,  ...,  0.7961,  0.7961,  0.7961],
          [-0.0588, -0.0431, -0.0196,  ...,  0.8039,  0.8039,  0.8039],
          [-0.0431, -0.0353, -0.0118,  ...,  0.8039,  0.8039,  0.8039],
          ...,
          [-0.8039, -0.8118, -0.8431,  ..., -0.5059, -0.4902, -0.4902],
          [-0.8118, -0.8196, -0.8510,  ..., -0.5137, -0.4980, -0.4902],
          [-0.8118, -0.8275, -0.8588,  ..., -0.5137, -0.5059, -0.4980]],

         [[-0.0118, -0.0039,  0.0275,  ...,  0.6627,  0.6627,  0.6627],
          [-0.0039,  0.0118,  

In [ ]:
real_A.size(0)

1

In [ ]:
D_A.output_shape

(1, 16, 16)

In [ ]:
valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)

In [ ]:
valid.shape

torch.Size([1, 1, 16, 16])

In [ ]:
valid

tensor([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
          [1

In [ ]:
real_A = Variable(batch["A"].type(Tensor))    # 사람 이미지 배치사이즈만큼
real_B = Variable(batch["B"].type(Tensor))    # 애니 애미지 배치사이즈만큼

valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)    # 1로 채워진 [1,1,16,16] 텐서 생성. discriminator의 label
fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)    # 1로 채워진 [1,1,16,16] 텐서 생성. discriminator의 label

G_AB.train()    # Generator 학습모드 전환
G_BA.train()    # Generator 학습모드 전환

optimizer_G.zero_grad()    # 미분을 통해 얻은 기울기를 0으로 초기화. 새로운 기울기를 구할 수 있음 

# (4) Identity loss : 색감, 형태 등을 유지하기 위한 값
loss_id_A = criterion_identity(G_BA(real_A), real_A)    # 실제 사람 이미지를 가지고 생성한 이미지와 실제 사람 이미지를 비교하여 L1 loss 계산
loss_id_B = criterion_identity(G_AB(real_B), real_B)    # 실제 애니 이미지를 가지고 생성한 이미지와 실제 애니 이미지를 비교하여 L1 loss 계산

loss_identity = (loss_id_A + loss_id_B) / 2    # 위 값의 평균이 identitiy loss값


# (5) GAN loss
fake_B = G_AB(real_A)    # 실제 사람 이미지로 생성한 애니 이미지
loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)    # 생성된 애니 이미지를 진짜인지 가짜인지 분류 후 MSE loss 계산
fake_A = G_BA(real_B)    # 실제 애니 이미지로 생성한 사람 이미지
loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)    # 생성된 사람 이미지를 진짜인지 가짜인지 분류 후 MSE loss 계산

loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2    # 위 값의 평균이 GAN loss값

In [ ]:
D_B(fake_B).shape

torch.Size([1, 1, 16, 16])

In [ ]:
D_B(fake_B)

tensor([[[[-0.5385, -0.2609, -0.6837, -0.5663, -1.2807, -0.4364,  0.4466,
           -1.5307, -1.2009, -0.9293, -0.4723, -0.9122, -1.0933, -0.7185,
           -0.5515, -0.4422],
          [-0.9425, -0.8221, -0.9328, -2.5480, -0.8875, -0.4895,  0.8747,
           -0.9124, -1.9631,  0.0663,  0.0105, -1.9702, -0.9078, -0.7634,
            0.1609, -0.4748],
          [-1.0496, -0.1590,  0.1700, -1.1443, -1.1056, -1.3435, -1.0812,
           -2.2141, -2.4557, -0.6571,  0.2358, -0.3887, -1.3687,  0.8638,
           -0.9736, -1.5514],
          [-0.8553, -0.1949, -2.7262, -0.1439, -3.7551, -0.2114, -0.5977,
           -1.8524, -0.6225,  1.1741, -0.8058,  0.4506,  0.5779, -0.1379,
           -3.4818, -0.3863],
          [ 0.8640,  0.8441, -2.3090, -1.6754, -1.8631,  0.3730, -3.4543,
           -2.4935, -0.9957, -2.0284, -2.2744, -3.7189, -0.3889, -2.7666,
            0.0780, -1.7351],
          [-1.7201, -2.5406, -2.4832, -1.3070,  0.4801,  0.7282, -0.2209,
           -0.1875, -2.5636, -1.0267

In [ ]:
D_A(real_A).shape

torch.Size([1, 1, 16, 16])

In [ ]:
D_A(real_A)

tensor([[[[-1.0250, -1.1591, -1.0351, -0.1347, -0.4656, -0.5077, -1.0215,
           -1.0815, -1.0431, -0.4294, -0.0719, -0.2711, -1.5414, -1.7401,
           -1.8943, -0.6797],
          [-1.8457, -1.9872, -0.8827,  0.8154,  1.1164,  0.5998, -0.7189,
            0.1315,  0.1571,  0.9007,  0.8999,  2.1333, -0.5298, -1.5339,
           -1.0321, -0.8263],
          [-1.5336, -4.7491, -1.2654, -0.3096,  0.4432, -0.6665, -0.4688,
            0.6947,  1.1449,  0.9774,  2.1416,  1.5974,  0.5861, -0.9467,
           -2.6035, -1.3595],
          [-1.5767, -3.7998, -1.0460,  0.0485,  0.7825, -0.5068, -0.4468,
            1.6961,  1.0006,  0.8205,  0.1018,  1.1686,  2.0836, -0.1198,
           -2.5900, -1.8483],
          [-1.2662, -1.9109, -0.7958, -2.3081,  0.1467, -0.8071, -1.6295,
           -0.5807,  1.2267, -0.4077, -0.1969, -2.0713,  0.2089,  0.4039,
           -0.8898, -2.0715],
          [-1.2845, -1.8262, -1.6859, -0.3791, -2.8172, -1.8922, -0.2076,
           -1.2080,  1.4720, -0.7473

In [ ]:
prev_time = time.time()
for epoch in range(init_epoch, n_epochs):
    for i, batch in enumerate(dataloader):

        # (1) Set model input
        real_A = Variable(batch["A"].type(Tensor))    # 사람 이미지 배치사이즈만큼
        real_B = Variable(batch["B"].type(Tensor))    # 애니 애미지 배치사이즈만큼


        # (2) Adversarial ground truths
        valid = Variable(Tensor(np.ones((real_A.size(0), *D_A.output_shape))), requires_grad=False)    # 1로 채워진 [1,1,16,16] 텐서 생성. discriminator의 label
        fake = Variable(Tensor(np.zeros((real_A.size(0), *D_A.output_shape))), requires_grad=False)    # 1로 채워진 [1,1,16,16] 텐서 생성. discriminator의 label


        # (3) Train Generators
        G_AB.train()    # Generator 학습모드 전환
        G_BA.train()    # Generator 학습모드 전환

        optimizer_G.zero_grad()    # Generator의 미분을 통해 얻은 기울기를 0으로 초기화. 새로운 기울기를 구할 수 있음 


        # (4) Identity loss : 색감, 형태 등을 유지하기 위한 값
        loss_id_A = criterion_identity(G_BA(real_A), real_A)    # 실제 사람 이미지를 가지고 생성한 이미지와 실제 사람 이미지를 비교하여 L1 loss 계산
        loss_id_B = criterion_identity(G_AB(real_B), real_B)    # 실제 애니 이미지를 가지고 생성한 이미지와 실제 애니 이미지를 비교하여 L1 loss 계산

        loss_identity = (loss_id_A + loss_id_B) / 2    # 위 값의 평균이 identitiy loss값


        # (5) GAN loss
        fake_B = G_AB(real_A)    # 실제 사람 이미지로 생성한 애니 이미지
        loss_GAN_AB = criterion_GAN(D_B(fake_B), valid)    # 생성된 애니 이미지를 진짜인지 가짜인지 분류 후 MSE loss 계산
        fake_A = G_BA(real_B)    # 실제 애니 이미지로 생성한 사람 이미지
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)    # 생성된 사람 이미지를 진짜인지 가짜인지 분류 후 MSE loss 계산

        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2    # 위 값의 평균이 GAN loss값


        # (6) Cycle loss
        recov_A = G_BA(fake_B)    # 생성된 애니 이미지로 다시 사람 이미지를 생성
        loss_cycle_A = criterion_cycle(recov_A, real_A)    # 위과정에서 생성된 이미지와 원본 사람 이미지를 비교하여 L1 loss 계산
        recov_B = G_AB(fake_A)    # 생성된 사람 이미지로 다시 애니 이미지를 생성
        loss_cycle_B = criterion_cycle(recov_B, real_B)    # 위 과정에서 생성된 이미지와 원본 애니 이미지를 비교하여 L1 loss 계산

        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2    # 위 값의 평균이 cycle loss 값


        # (7) Total loss
        loss_G = loss_GAN + lambda_cyc * loss_cycle + lambda_id * loss_identity    # Generator의 전체 loss 값은 GAN의 loss값 + cycle lambda*cycle loss + identity lambda*identity loss

        loss_G.backward()    # loss 값에 파라미터에 대한 기울기를 계산
        optimizer_G.step()    # 가중치 Update


        # (8) Train Discriminator A
        optimizer_D_A.zero_grad()    # Discriminator_A의 가중치 업데이트를 위한 기울기 초기화


        # (9) Real loss
        loss_real = criterion_GAN(D_A(real_A), valid)    # Discriminator_A가 실제 사람이미지를 진짜라고 판단한 경우 MSE 손실함수 계산


        # (10) Fake loss (on batch of previously generated samples)
        fake_A_ = fake_A_buffer.push_and_pop(fake_A)    # 생성된 사람 이미지를 따로 저장(Cycle GAN 학습 진행을 위해)
        loss_fake = criterion_GAN(D_A(fake_A_.detach()), fake)    # Discriminator_A가 생성 사람이미지를 가짜라고 판단한 경우 MSE 손실함수 계산. fake_A의 가중치는 업데이트 가능한 상태이므로 detach()함수로 값만 복사
        

        # (11) Total loss
        loss_D_A = (loss_real + loss_fake) / 2    # #9, #10에서 계산한 손실값의 평균

        loss_D_A.backward()    # 기울기 계산
        optimizer_D_A.step()    # 업데이트


        # (12) Train Discriminator B

        optimizer_D_B.zero_grad()    # Discriminator_B의 가중치 업데이트를 위한 기울기 초기화


        # (13) Real loss
        loss_real = criterion_GAN(D_B(real_B), valid)    # Discriminator_B가 실제 애니이미지를 진짜라고 판단한 경우 MSE 손실함수 계산


        # (14) Fake loss (on batch of previously generated samples)
        fake_B_ = fake_B_buffer.push_and_pop(fake_B)    # 생성된 애니이미지를 따로 저장
        loss_fake = criterion_GAN(D_B(fake_B_.detach()), fake)    # Discriminator_B가 생성 애니이미지를 가짜라고 판단한 경우 MSE 손실함수 계산


        # (15) Total loss
        loss_D_B = (loss_real + loss_fake) / 2    # #13, #14에서 계산한 손실값의 평균

        loss_D_B.backward()    # 기울기 계산
        optimizer_D_B.step()    # 업데이트

        loss_D = (loss_D_A + loss_D_B) / 2    # Discriminator 전체의 손실값


        # (16) Determine approximate time left
        batches_done = epoch * len(dataloader) + i    # 한 epoch에서 진행된 배치 수
        batches_left = n_epochs * len(dataloader) - batches_done    # 한 epoch에서 남은 배치 수
        time_left = datetime.timedelta(seconds=batches_left * (time.time() - prev_time))    # 남은 시간
        prev_time = time.time()


        # (17) Print log
        sys.stdout.write(
            "\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f, adv: %f, cycle: %f, identity: %f] ETA: %s"
            % (
                epoch,
                n_epochs,
                i,
                len(dataloader),
                loss_D.item(),
                loss_G.item(),
                loss_GAN.item(),
                loss_cycle.item(),
                loss_identity.item(),
                time_left,
            )
        )


        # (18) If at sample interval save image
        if batches_done % sample_interval == 0:    # 배치가 100번 진행될 때마다
            sample_images(batches_done)    # 이미지 저장합시당


    # (19) Update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()


    # (20) Save model checkpoints : 모델 가중치 저장
    if opt.checkpoint_interval != -1 and epoch % opt.checkpoint_interval == 0:    # checkpoint_interval이 -1이 아니고 epoch을 나눈 나머지가 0인 경우.. 즉, epoch 5번마다
        torch.save(G_AB.state_dict(), "saved_models/%s/G_AB_%d.pth" % (dataset_name, epoch))    # 모델 저장.. 밑도 동일
        torch.save(G_BA.state_dict(), "saved_models/%s/G_BA_%d.pth" % (dataset_name, epoch))
        torch.save(D_A.state_dict(), "saved_models/%s/D_A_%d.pth" % (dataset_name, epoch))
        torch.save(D_B.state_dict(), "saved_models/%s/D_B_%d.pth" % (dataset_name, epoch))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Epoch 19/20] [Batch 3399/3400] [D loss: 0.113236] [G loss: 2.150472, adv: 0.693580, cycle: 0.108289, identity: 0.074800] ETA: 0:00:00.319763